# Advisors

Advisors are an interesting feature in Spring-AI that allows you to flexibly intercept,
modify, and enhance AI interactions.

With `Advisors`, you can:
- Add necessary context to user requests
- Filter out harmful or sensitive content in AI requests
- Track custom metrics
- Ensure consistent output structure
- And more

Let's add dependencies and create a `ChatModel`

In [1]:
@file:DependsOn("org.springframework.ai:spring-ai-openai-spring-boot-starter:1.0.0-M6")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.18.2")

In [2]:
import org.springframework.ai.chat.client.ChatClient
import org.springframework.ai.openai.OpenAiChatModel
import org.springframework.ai.openai.OpenAiChatOptions
import org.springframework.ai.openai.api.OpenAiApi

val apiKey = System.getenv("OPENAI_API_KEY") ?: "YOUR_OPENAI_API_KEY"

val openAiApi = OpenAiApi.builder().apiKey(apiKey).build()
val openAiOptions = OpenAiChatOptions.builder()
    .model(OpenAiApi.ChatModel.GPT_4_O_MINI)
    .temperature(0.7)
    .build()


val chatModel = OpenAiChatModel.builder()
    .openAiApi(openAiApi)
    .defaultOptions(openAiOptions)
    .build()

Let's add the `MessageChatMemory` advisor.
As the name suggests, this advisor will implement message history,
preserving the conversation context.
For this Advisor, we'll need a `ChatMemory` instance
where messages will be stored.

In [3]:
import org.springframework.ai.chat.client.advisor.MessageChatMemoryAdvisor
import org.springframework.ai.chat.memory.ChatMemory
import org.springframework.ai.chat.memory.InMemoryChatMemory

val chatMemory = InMemoryChatMemory()

val chatClient = ChatClient
    .builder(chatModel)
    .defaultAdvisors(MessageChatMemoryAdvisor(chatMemory))
    .build()


Let's test how this works

In [4]:
chatClient.prompt("Hi, tell me a joke").call().content()

Sure! Why did the scarecrow win an award?

Because he was outstanding in his field!

In [5]:
chatClient.prompt("What is previous message in our chat history?").call().content()

The previous message in our chat history was a joke: "Why did the scarecrow win an award? Because he was outstanding in his field!"

As we can see, the LLM now has access to our message history.

Spring-AI includes several predefined Advisors:
- `MessageChatMemoryAdvisor`
- `PromptChatMemoryAdvisor`
- `QuestionAnswerAdvisor`
- `RetrievalAugmentationAdvisor`
- `SafeGuardAdvisor`
- `SimpleLoggerAdvisor`
- `VectorStoreChatMemoryAdvisor`

And you can create your own custom Advisor as well.

Let's do that now.
We'll create an Advisor that logs requests and responses by outputting them to our console.
To do this, we'll extend `CallAroundAdvisor` and implement the `aroundCall` method

In [6]:
import org.springframework.ai.chat.client.advisor.api.AdvisedRequest
import org.springframework.ai.chat.client.advisor.api.AdvisedResponse
import org.springframework.ai.chat.client.advisor.api.CallAroundAdvisor
import org.springframework.ai.chat.client.advisor.api.CallAroundAdvisorChain

class CustomLogger: CallAroundAdvisor {
    override fun getName(): String {
        return "CustomLogger"
    }

    override fun getOrder(): Int = 0

    override fun aroundCall(advisedRequest: AdvisedRequest, chain: CallAroundAdvisorChain): AdvisedResponse {
        println("CustomLogger.Before: ${advisedRequest}")
        val advisedResponse = chain.nextAroundCall(advisedRequest)
        println("CustomLogger.After: ${advisedResponse}")
        return advisedResponse
    }
}

Now let's see our `CustomAdvisor` in action

In [7]:
chatClient
    .prompt("Generate HelloWorld in Kotlin")
    .advisors(CustomLogger())
    .call()
    .content()

CustomLogger.Before: AdvisedRequest[chatModel=OpenAiChatModel [defaultOptions=OpenAiChatOptions: {"streamUsage":false,"model":"gpt-4o-mini","temperature":0.7}], userText=Generate HelloWorld in Kotlin, systemText=null, chatOptions=OpenAiChatOptions: {"streamUsage":false,"model":"gpt-4o-mini","temperature":0.7}, media=[], functionNames=[], functionCallbacks=[], messages=[UserMessage{content='Hi, tell me a joke', properties={messageType=USER}, messageType=USER}, AssistantMessage [messageType=ASSISTANT, toolCalls=[], textContent=Sure! Why did the scarecrow win an award?

Because he was outstanding in his field!, metadata={finishReason=STOP, refusal=, index=0, role=ASSISTANT, id=chatcmpl-BGAEb420M9SFn16wamX1DVRevyAS6, messageType=ASSISTANT}], UserMessage{content='What is previous message in our chat history?', properties={messageType=USER}, messageType=USER}, AssistantMessage [messageType=ASSISTANT, toolCalls=[], textContent=The previous message in our chat history was a joke: "Why did the 

Sure! Here's a simple "Hello, World!" program in Kotlin:

```kotlin
fun main() {
    println("Hello, World!")
}
```

You can run this code in any Kotlin environment or IDE, and it will print "Hello, World!" to the console.